In [6]:
import pandas as pd
from sudachipy import tokenizer
from sudachipy import dictionary
from wanakana import to_hiragana, is_japanese, is_katakana, is_hiragana, is_kanji
from deep_translator import (ChatGptTranslator, GoogleTranslator)
import string

In [3]:
tokenizer_obj = dictionary.Dictionary(dict_type="small").create()

KANJI_READING_MAPPING = {
    '私': '私[わたし]',
    '貴女': '貴女[あなた]',
    '何': '何[なに]',
    '外宇宙': '外宇宙[がいうちゅう]',
    '異星人': '異星人[いせいじん]',
    '優那': '優那[ゆうな]',
    '菜々美': '菜々美[ななみ]'
}

JAPANESE_PUNCTUATION = '　〜！？。、（）：「」『』０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'

SPECIAL_CHARACTERS = '〜'

def is_japanese_extended(text):
    return is_japanese(text) and text not in string.punctuation and text not in JAPANESE_PUNCTUATION

def to_anki_format(index, kanji, reading):
    return '{}{}[{}]'.format(' ' if index > 0 else '', kanji, reading) 

def add_furigana(text):
    tokens = [m for m in tokenizer_obj.tokenize(text, tokenizer.Tokenizer.SplitMode.C)]
    parsed = ''
    token_indexes_to_skip = []
    for index, token in enumerate(tokens):   
        if index in token_indexes_to_skip:
          continue
        to_parse = is_japanese_extended(token.surface()) and not is_katakana(token.surface()) and not is_hiragana(token.surface())
        if to_parse:
            if token.surface()[-1] in SPECIAL_CHARACTERS:
                parsed += add_furigana(token.surface()[:-1]) + token.surface()[-1]
            else:
                if index > 0:
                    parsed += ' '
                reading = to_hiragana(token.reading_form())
                if token.surface() in KANJI_READING_MAPPING:
                    parsed += KANJI_READING_MAPPING[token.surface()]
                elif index < len(tokens)-1 and token.surface() + tokens[index+1].surface() in KANJI_READING_MAPPING:
                    parsed += KANJI_READING_MAPPING[tokens[index].surface() + tokens[index+1].surface()]
                    token_indexes_to_skip.append(index+1)
                else:
                    surface_index = 0
                    reading_index = 0
                    while len(token.surface()) > surface_index:
                        if is_hiragana(token.surface()[surface_index]) or is_katakana(token.surface()[surface_index]):
                            parsed += token.surface()[surface_index]
                            reading_index += 1
                            surface_index += 1
                        else:
                            next_index = -1
                            for token_index in range(surface_index, len(token.surface())):
                                if is_hiragana(token.surface()[token_index]) or is_katakana(token.surface()[token_index]):
                                    next_index = token_index
                                    break
                            if next_index < 0:
                                parsed += to_anki_format(
                                  index=surface_index, 
                                  kanji=token.surface()[surface_index:], reading=reading[reading_index:])
                                break
                            else:
                                reading_index_tail = reading_index
                                while reading[reading_index_tail] != token.surface()[next_index] or (reading_index_tail < len(reading)-1 and reading[reading_index_tail] == reading[reading_index_tail+1]):
                                    reading_index_tail += 1
                                parsed += to_anki_format(
                                  index=surface_index, 
                                  kanji=token.surface()[surface_index:next_index], reading=reading[reading_index:reading_index_tail])
                                reading_index = reading_index_tail
                            reading_length = next_index - surface_index
                            if reading_length > 0:
                                surface_index += reading_length
                            else:
                                break
        else:
            parsed += token.surface()
    return parsed

/var/folders/vt/st5d0xqj4vn9bjz3rgc4jhqc0000gn/T/ipykernel_34142/3941344110.py:1: DeprecationWarning: Parameter dict_type of Dictionary() is deprecated, use dict instead
  tokenizer_obj = dictionary.Dictionary(dict_type="small").create()


In [50]:
def translate(word):
    return GoogleTranslator(source='japanese', target='english').translate(text=word)

In [57]:
vocabulary = pd.read_csv('new_vocab.csv')
vocabulary

,Vocab,Sentence
0,国会議事堂,国会議事堂は見学したいんですが、どうしたらいいですか
1,見学,国会議事堂は見学したいんですが、どうしたらいいですか
2,直接,直接いったらいいですよ
3,引っ越す,ミラーさん引っ越しの荷物は片づきました
4,荷物,ミラーさん引っ越しの荷物は片づきました
5,片づく,ミラーさん引っ越しの荷物は片づきました
6,燃える,燃えるごみは月曜日朝出してください
7,駐車場,ごみ置き場は駐車場の横です
8,瓶,瓶や缶はいつですか
9,缶,瓶や缶はいつですか


In [58]:
data = []
columns = ['index', 'word', 'word_with_reading', 'definition', 'example_sentence', 'sentence_with_reading', 'sentence_translation', 'Kanji', 'v1', 'word_audio', 'setence_audio']
df = pd.DataFrame(data, columns=columns)

In [59]:
df['index'] = vocabulary['Vocab']
df['word'] = vocabulary['Vocab']
df['word_with_reading'] = vocabulary['Vocab'].apply(add_furigana)
df['definition'] = vocabulary['Vocab'].apply(translate)
df['example_sentence'] = vocabulary['Sentence']
df['sentence_with_reading'] = vocabulary['Sentence'].apply(add_furigana)
df['sentence_translation'] = vocabulary['Sentence'].apply(translate)
df['Kanji'] = True
df

,index,word,word_with_reading,definition,example_sentence,sentence_with_reading,sentence_translation,Kanji,v1,word_audio,setence_audio
0,国会議事堂,国会議事堂,国会[こっかい] 議事[ぎじ] 堂[どう],Houses of Parliament,国会議事堂は見学したいんですが、どうしたらいいですか,国会[こっかい] 議事[ぎじ] 堂[どう]は 見学[けんがく]したいんですが、どうしたらいいですか,I would like to tour the National Diet Buildin...,True,NaN,NaN,NaN
1,見学,見学,見学[けんがく],visit,国会議事堂は見学したいんですが、どうしたらいいですか,国会[こっかい] 議事[ぎじ] 堂[どう]は 見学[けんがく]したいんですが、どうしたらいいですか,I would like to tour the National Diet Buildin...,True,NaN,NaN,NaN
2,直接,直接,直接[ちょくせつ],directly,直接いったらいいですよ,直接[ちょくせつ]いったらいいですよ,It's better if you go directly,True,NaN,NaN,NaN
3,引っ越す,引っ越す,引[ひ]っ 越[こ]す,move,ミラーさん引っ越しの荷物は片づきました,ミラーさん 引[ひ]っ 越[こ]しの 荷物[にもつ]は 片[かた]づきました,Mr. Miller has packed up his moving belongings.,True,NaN,NaN,NaN
4,荷物,荷物,荷物[にもつ],luggage,ミラーさん引っ越しの荷物は片づきました,ミラーさん 引[ひ]っ 越[こ]しの 荷物[にもつ]は 片[かた]づきました,Mr. Miller has packed up his moving belongings.,True,NaN,NaN,NaN
5,片づく,片づく,片[かた]づく,Tidy up,ミラーさん引っ越しの荷物は片づきました,ミラーさん 引[ひ]っ 越[こ]しの 荷物[にもつ]は 片[かた]づきました,Mr. Miller has packed up his moving belongings.,True,NaN,NaN,NaN
6,燃える,燃える,燃[も]える,burn,燃えるごみは月曜日朝出してください,燃[も]えるごみは 月曜[げつよう] 日[ひ] 朝[あさ] 出[だ]してください,Please put out your burnable garbage on Monday...,True,NaN,NaN,NaN
7,駐車場,駐車場,駐車[ちゅうしゃ] 場[じょう],parking,ごみ置き場は駐車場の横です,ごみ 置[お]き 場[ば]は 駐車[ちゅうしゃ] 場[じょう]の 横[よこ]です,Garbage storage is next to the parking lot,True,NaN,NaN,NaN
8,瓶,瓶,瓶[びん],bottle,瓶や缶はいつですか,瓶[びん]や 缶[かん]はいつですか,When are bottles and cans available?,True,NaN,NaN,NaN
9,缶,缶,缶[かん],can,瓶や缶はいつですか,瓶[びん]や 缶[かん]はいつですか,When are bottles and cans available?,True,NaN,NaN,NaN


In [60]:
df.to_csv('vocab.csv', encoding="utf-8", index=False, header=False)